In [4]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry
import openmeteo_requests

import requests_cache
from retry_requests import retry

from shapely import wkt
from shapely.geometry import Point
import geopandas as gpd

import warnings

warnings.filterwarnings('ignore')

from pyproj import Proj, transform
import datetime as dt


In [5]:
file_path = 'meshblocks-auckland-higher.csv'
meshblock_data = pd.read_csv(file_path)

# Parse the WKT column to extract geometries
meshblock_data['geometry'] = meshblock_data['WKT'].apply(wkt.loads)

# Convert to a GeoDataFrame
gdf = gpd.GeoDataFrame(meshblock_data, geometry='geometry')

# Set the initial CRS to EPSG:2193 (modify this if different)
gdf.set_crs(epsg=2193, inplace=True)

# Convert to WGS 84 (latitude and longitude)
gdf = gdf.to_crs(epsg=4326)

# Calculate centroids for each geometry
gdf['centroid'] = gdf['geometry'].centroid

# Extract latitude and longitude from centroids
gdf['latitude'] = gdf['centroid'].apply(lambda x: x.y)
gdf['longitude'] = gdf['centroid'].apply(lambda x: x.x)

# Display the data with centroids
meshblock_centroids = gdf[['MB2024_V1_00', 'latitude', 'longitude', 'SA12023_V1_00', 'SA22023_V1_00', 'SA32023_V1_00']]
meshblock_centroids.head()


,MB2024_V1_00,latitude,longitude,SA12023_V1_00,SA22023_V1_00,SA32023_V1_00
0,4019145,-36.899286,174.716277,7037220,135500,51440
1,4001328,-36.852361,174.777485,7037219,136400,51200
2,827902,-37.218339,174.943047,7032417,169901,52450
3,4014920,-37.100411,174.946908,7032175,164302,52250
4,4004236,-37.093404,174.951154,7010152,164302,52250


In [6]:
# Aggregate centroids by Mesh MB2022_V1_00
mesh_centroid = meshblock_centroids.groupby('MB2024_V1_00').agg({
    'latitude': 'mean',
    'longitude': 'mean'
}).reset_index()
mesh_centroid.head()

# Aggregate centroids by SA12022
sa1_centroids = meshblock_centroids.groupby('SA12023_V1_00').agg({
    'latitude': 'mean',
    'longitude': 'mean'
}).reset_index()

# Aggregate centroids by SA22022
sa2_centroids = meshblock_centroids.groupby('SA22023_V1_00').agg({
    'latitude': 'mean',
    'longitude': 'mean'
}).reset_index()

# Aggregate centroids by SA22022
sa3_centroids = meshblock_centroids.groupby('SA32023_V1_00').agg({
    'latitude': 'mean',
    'longitude': 'mean'
}).reset_index()

# Display aggregated centroids
#mesh_centroid.head(),sa1_centroids.head(), sa2_centroids.head()


## Set SA3 code, date and light condition

In [7]:
light_condition = 'Day'
#setdate = '2024-05-28'
sa3_code = 51440 # mount albert 

In [8]:
#find SA2_coordinates
SA3_coordinates = sa3_centroids[sa3_centroids['SA32023_V1_00'] == sa3_code][['latitude', 'longitude']]

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": SA3_coordinates['latitude'],
	"longitude": SA3_coordinates['longitude'],
	"daily": ["weather_code", "temperature_2m_min", "wind_speed_10m_max"],
	"timezone": "Pacific/Auckland",
	"forecast_days": 14
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Current values. The order of variables needs to be the same as requested.

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max

daily_dataframe = pd.DataFrame(data = daily_data)

# Define mapping functions for weatherA, based on weather_code from open-meteo and filtered date
def map_to_weatherA(row):
    if row['weather_code'] in [71, 73, 75, 77, 85, 86]:
        return 'Snow'
    elif row['weather_code'] in [63, 65, 81, 82]:
        return 'Heavy rain'
    elif row['weather_code'] in [51, 53, 55, 56, 57, 61, 80]:
        return 'Light rain'
    elif row['weather_code'] in [66, 67, 96, 99]:
        return 'Hail or Sleet'
    elif row['weather_code'] in [45, 48]:
        return 'Mist or Fog'
    else:
        return 'Fine'

#for weatherB, based on wind speed and temperature. 
def map_to_weatherB(row):
    if row['wind_speed_10m_max'] > 30:
        return 'Strong wind'
    elif row['temperature_2m_min'] <= 0:
        return 'Frost'
    else:
        return 'None'

# Apply the mapping functions
daily_dataframe['weatherA'] = daily_dataframe.apply(map_to_weatherA, axis=1)
daily_dataframe['weatherB'] = daily_dataframe.apply(map_to_weatherB, axis=1)
daily_dataframe['light'] = light_condition
# extract date only
daily_dataframe['date'] = pd.to_datetime(daily_dataframe['date'])
daily_dataframe['date'] = daily_dataframe['date'].dt.date

# filter by set date 
#filtered_df = daily_dataframe[daily_dataframe['date'].dt.date == pd.Timestamp(setdate).date()]
#filtered_df = daily_dataframe
result_day = daily_dataframe[['date', 'weatherA', 'weatherB', 'light']]


result_night = daily_dataframe.copy()
result_night['light'] = result_night['light'].replace('Day', 'Night')
result_night = result_night[['date', 'weatherA', 'weatherB', 'light']]
#print(result_day)
#print(result_night)
combined_result = pd.concat([result_day, result_night], ignore_index=True)

# Sort the DataFrame by 'date' and then by 'light' to ensure 'Day' comes before 'Night'
combined_result.sort_values(by=['date'], ascending=[True], inplace=True)

# Reset index after sorting
combined_result.reset_index(drop=True, inplace=True)

combined_result.to_csv('weather_forecaset.csv', index=False)
combined_result

Coordinates -36.875°N 174.75°E
Elevation 57.0 m asl
Timezone b'Pacific/Auckland' b'NZST'
Timezone difference to GMT+0 43200 s


,date,weatherA,weatherB,light
0,2024-05-27,Fine,None,Day
1,2024-05-27,Fine,None,Night
2,2024-05-28,Light rain,Strong wind,Day
3,2024-05-28,Light rain,Strong wind,Night
4,2024-05-29,Fine,Strong wind,Day
5,2024-05-29,Fine,Strong wind,Night
6,2024-05-30,Fine,None,Day
7,2024-05-30,Fine,None,Night
8,2024-05-31,Fine,None,Day
9,2024-05-31,Fine,None,Night


In [9]:
combined_result.to_csv('weather_forecaset.csv', index=False)